In [1]:
import os
import gzip
import shutil
import pandas as pd
from tqdm import tqdm

In [2]:
DATA_DIR = 'D:/species_classifier/data/inaturalist/'
AWS_BASE = 'aws s3 --no-sign-request --region us-east-1 cp s3://inaturalist-open-data/'

### Downloading Data Files

In [4]:
!aws --version

aws-cli/1.22.28 Python/3.7.7 Windows/10 botocore/1.23.28


File association not found for extension .py


In [5]:
os.system(AWS_BASE + 'observations.csv.gz ' + DATA_DIR + 'observations.csv.gz')

0

In [6]:
os.system(AWS_BASE + 'photos.csv.gz ' + DATA_DIR + 'photos.csv.gz')

0

In [7]:
os.system(AWS_BASE + 'taxa.csv.gz ' + DATA_DIR + 'taxa.csv.gz')

0

### Unzipping Data Files

In [11]:
with gzip.open(DATA_DIR + 'observations.csv.gz', 'rb') as f_in:
    with open(DATA_DIR + 'observations.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
os.remove(DATA_DIR + 'observations.csv.gz')

In [12]:
with gzip.open(DATA_DIR + 'photos.csv.gz', 'rb') as f_in:
    with open(DATA_DIR + 'photos.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
os.remove(DATA_DIR + 'photos.csv.gz')

In [10]:
with gzip.open(DATA_DIR + 'taxa.csv.gz', 'rb') as f_in:
    with open(DATA_DIR + 'taxa.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
os.remove(DATA_DIR + 'taxa.csv.gz')

### Create Unified File

In [3]:
def process_partition(partition):
    photo_keys = {}
    observation_keys = {}
    observation_dict = {}
    par_len = len(partition)
    
    with open(DATA_DIR + 'observations.csv', 'r') as observations_file:
        first_line = True
        for observation_line in observations_file:
            if first_line:
                observation_line = observation_line.replace('\r', '').replace('\n', '').split('\t')
                for i in range(len(observation_line)):
                    observation_keys[observation_line[i]] = i
                first_line = False
                continue
            observation_line = observation_line.replace('\r', '').replace('\n', '').split('\t')
            
            if observation_line[observation_keys['observation_uuid']][-1*par_len:] == partition:
                observation_dict[observation_line[observation_keys['observation_uuid']]] = {
                    'taxon_id': observation_line[observation_keys['taxon_id']],
                    'photos': []
                }
                
    with open(DATA_DIR + 'photos.csv', 'r') as photos_file:
        first_line = True
        for photo_line in photos_file:
            if first_line:
                photo_line = photo_line.replace('\r', '').replace('\n', '').split('\t')
                for i in range(len(photo_line)):
                    photo_keys[photo_line[i]] = i
                first_line = False
                continue
            photo_line = photo_line.replace('\r', '').replace('\n', '').split('\t')
            
            if photo_line[photo_keys['observation_uuid']][-1*par_len:] == partition:
                try:
                    observation_dict[photo_line[photo_keys['observation_uuid']]]['photos'].append((photo_line[photo_keys['photo_id']], photo_line[photo_keys['extension']]))
                except KeyError as e:
                    pass
    
    output_count = 0
    print('\nPartition: ' + partition)
    print('\tObservations: ' + str(len(observation_dict)))
    with open(DATA_DIR + 'inaturalist_data.csv', 'a') as output_file:
        for observation_uuid in observation_dict:
            for photo in observation_dict[observation_uuid]['photos']:
                try:
                    output_line = photo[0] + '\t'
                    output_line += photo[1] + '\t'
                    output_line += observation_dict[observation_uuid]['taxon_id'] + '\n'
                    output_file.write(output_line)
                    output_count += 1
                except Exception as e:
                    pass
    print('\tPhotos Added: ' + str(output_count))

def create_output_data(partition_len=1):
    photo_keys = {}
    output_count = 0
    with open(DATA_DIR + 'inaturalist_data.csv', 'w') as output_file:
        output_file.write('')
    
    print('Getting Partitions...')
    partitions = set()
    with open(DATA_DIR + 'photos.csv', 'r') as photos_file:
        first_line = True
        for photo_line in photos_file:
            if first_line:
                photo_line = photo_line.replace('\r', '').replace('\n', '').split('\t')
                for i in range(len(photo_line)):
                    photo_keys[photo_line[i]] = i
                first_line = False
                continue
            photo_line = photo_line.replace('\r', '').replace('\n', '').split('\t')
            
            partitions.add(photo_line[photo_keys['observation_uuid']][-1*partition_len:])
    print('\tFound ' + str(len(list(partitions))) + ' Partitions')
    
    print('Iterating Through Partitions...')
    for partition in tqdm(partitions):
        process_partition(partition)

In [4]:
create_output_data()

Getting Partitions...


  0%|                                                                                           | 0/16 [00:00<?, ?it/s]
0it [00:00, ?it/s]
71731it [00:00, 714047.14it/s]

	Found 16 Partitions
Iterating Through Partitions...



147689it [00:00, 725540.30it/s]
226494it [00:00, 741691.27it/s]
304152it [00:00, 750849.05it/s]
381085it [00:00, 754628.22it/s]
457407it [00:00, 755484.03it/s]
524613it [00:00, 721246.01it/s]
600344it [00:00, 730818.73it/s]
677569it [00:00, 741160.62it/s]
751790it [00:01, 741044.84it/s]
825201it [00:01, 737217.90it/s]
901206it [00:01, 742123.73it/s]
980483it [00:01, 755241.82it/s]
1055824it [00:01, 752980.29it/s]
1130725it [00:01, 745666.85it/s]
1205030it [00:01, 744306.37it/s]
1279278it [00:01, 673724.53it/s]
1347802it [00:01, 670714.05it/s]
1425110it [00:01, 698209.84it/s]
1502249it [00:02, 718596.42it/s]
1580603it [00:02, 735526.65it/s]
1656583it [00:02, 740884.94it/s]
1734316it [00:02, 751025.26it/s]
1809764it [00:02, 744977.42it/s]
1888104it [00:02, 756023.42it/s]
1966556it [00:02, 762647.37it/s]
2042987it [00:02, 665067.86it/s]
2115310it [00:02, 680585.85it/s]
2186359it [00:03, 688200.29it/s]
2258015it [00:03, 695748.68it/s]
2334833it [00:03, 714979.89it/s]
2411411it [00:03, 728


Partition: e
	Observations: 5602190


...

	Photos Added: 9584854


133565it [00:00, 611419.12it/s]
211778it [00:00, 653956.05it/s]
286597it [00:00, 679318.33it/s]
343464it [00:00, 641359.84it/s]
424038it [00:00, 682516.03it/s]
504627it [00:00, 714971.20it/s]
583913it [00:00, 735007.44it/s]
662926it [00:00, 748566.21it/s]
742372it [00:01, 761314.26it/s]
817637it [00:01, 756765.54it/s]
892714it [00:01, 721127.51it/s]
968806it [00:01, 731531.83it/s]
1047274it [00:01, 745553.93it/s]
1128010it [00:01, 761602.37it/s]
1208526it [00:01, 773072.18it/s]
1289497it [00:01, 783011.55it/s]
1369826it [00:01, 787354.69it/s]
1449572it [00:01, 789738.09it/s]
1529284it [00:02, 791243.54it/s]
1608455it [00:02, 787223.74it/s]
1691128it [00:02, 796987.92it/s]
1770882it [00:02, 789415.18it/s]
1849879it [00:02, 787749.23it/s]
1928694it [00:02, 779164.90it/s]
2006660it [00:02, 735906.49it/s]
2085871it [00:02, 750422.79it/s]
2165751it [00:02, 763391.58it/s]
2246778it [00:02, 775214.91it/s]
2327871it [00:03, 784963.97it/s]
2409358it [00:03, 793073.63it/s]
2489566it [00:03, 7946


Partition: a
	Observations: 5598239


100%|███████████████████████████████████████████████████████████████████████████████| 16/16 [1:25:21<00:00, 320.10s/it]

	Photos Added: 9585535


### Filter for Animal Species

In [4]:
inaturalist_df = pd.read_csv(DATA_DIR + 'inaturalist_data.csv', delimiter='\t', header=None)
inaturalist_df = inaturalist_df.dropna()
inaturalist_df[0] = inaturalist_df[0].astype(int)
inaturalist_df[1] = inaturalist_df[1].astype(str)
inaturalist_df[2] = inaturalist_df[2].astype(int)
inaturalist_df

,0,1,2
0,21223,jpg,67438
1,21236,jpg,124171
2,21238,jpg,67559
3,21283,jpg,555970
4,21285,jpg,555970
...,...,...,...
153429239,249325291,jpg,453937
153429240,249325363,jpg,321690
153429241,249323740,jpeg,50822
153429242,249325544,jpg,48419


In [3]:
taxa_df = pd.read_csv(DATA_DIR + 'taxa.csv', delimiter='\t')
taxa_df = taxa_df[(taxa_df['rank'] == 'species') & taxa_df['active']] # Getting only Active Species
taxa_df = taxa_df[taxa_df['ancestry'].str.contains('/2/', na=False)] # Getting only Chordates
taxa_df

,taxon_id,ancestry,rank_level,rank,name,active
2,14866,48460/1/2/355675/3/7251/14841/14865,10.0,species,Gracula ptilogenys,True
3,11218,48460/1/2/355675/3/7251/10732/559715/979788,10.0,species,Cranioleuca erythrops,True
5,10773,48460/1/2/355675/3/7251/10732/559715/10768,10.0,species,Thripadectes holostictus,True
6,11624,48460/1/2/355675/3/7251/10732/559715/11613,10.0,species,Phacellodomus rufifrons,True
8,13857,48460/1/2/355675/3/7251/13685/13847,10.0,species,Passer pyrrhonotus,True
...,...,...,...,...,...,...
1393606,1424864,48460/1/2/355675/26036/26172/85552/35001/35002,10.0,species,Heloderma exasperatum,True
1393607,1424865,48460/1/2/355675/26036/26172/85552/35001/35002,10.0,species,Heloderma charlesbogerti,True
1393643,1424866,48460/1/2/355675/26036/26172/85552/35001/35002,10.0,species,Heloderma alvarezi,True
1394063,1428033,48460/1/2/355675/47178/49216/1303593/513686/90170,10.0,species,Nemacheilus cacao,True


In [7]:
taxon_id_set = set(list(taxa_df['taxon_id']))
inaturalist_df = inaturalist_df[inaturalist_df[2].isin(taxon_id_set)]
inaturalist_df

,0,1,2
7,21329,jpg,4328
8,21543,jpg,204452
9,21550,jpg,4665
10,21551,jpg,4665
11,21552,jpg,4665
...,...,...,...
153429169,249324626,jpeg,14886
153429178,249324706,jpeg,357
153429226,249325313,jpg,1416307
153429229,249325412,jpeg,4981


### Filter for Photo Count by Species

In [24]:
species_counts = inaturalist_df.groupby(2).count()[0].sort_values()
species_counts = species_counts[species_counts >= 1000]
species_counts

2
792951      1000
117570      1001
12711       1003
6643        1003
1081        1003
           ...  
42223     180888
4956      200795
13858     201496
5212      206921
6930      302345
Name: 0, Length: 2652, dtype: int64

In [25]:
taxon_id_set = set(list(species_counts.index))
inaturalist_df = inaturalist_df[inaturalist_df[2].isin(taxon_id_set)]
inaturalist_df

,0,1,2
7,21329,jpg,4328
9,21550,jpg,4665
10,21551,jpg,4665
11,21552,jpg,4665
12,21553,jpg,4665
...,...,...,...
153429168,249324554,jpeg,46017
153429169,249324626,jpeg,14886
153429178,249324706,jpeg,357
153429229,249325412,jpeg,4981


In [29]:
inaturalist_df.to_csv(DATA_DIR + 'inaturalist_data.csv', sep='\t', header=False, index=False) 

### Train/Test Split